In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Get Dataset

In [2]:
test_df=pd.read_excel('../Data/Test_set.xlsx')
train_df=pd.read_excel('../Data/Data_Train.xlsx')

In [3]:
test_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [4]:
train_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [5]:
big_df=train_df.append(test_df,sort=False)

In [6]:
big_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0


In [7]:
big_df.tail()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
2666,Air India,6/06/2019,Kolkata,Banglore,CCU → DEL → BLR,20:30,20:25 07 Jun,23h 55m,1 stop,No info,NaN
2667,IndiGo,27/03/2019,Kolkata,Banglore,CCU → BLR,14:20,16:55,2h 35m,non-stop,No info,NaN
2668,Jet Airways,6/03/2019,Delhi,Cochin,DEL → BOM → COK,21:50,04:25 07 Mar,6h 35m,1 stop,No info,NaN
2669,Air India,6/03/2019,Delhi,Cochin,DEL → BOM → COK,04:00,19:15,15h 15m,1 stop,No info,NaN
2670,Multiple carriers,15/06/2019,Delhi,Cochin,DEL → BOM → COK,04:55,19:15,14h 20m,1 stop,No info,NaN


In [8]:
test_df.shape

(2671, 10)

In [9]:
train_df.shape

(10683, 11)

In [10]:
big_df.dtypes

Airline             object
Date_of_Journey     object
Source              object
Destination         object
Route               object
Dep_Time            object
Arrival_Time        object
Duration            object
Total_Stops         object
Additional_Info     object
Price              float64
dtype: object

# Check Missing Value

In [11]:
df_dtypes = pd.merge(big_df.isnull().sum(axis = 0).sort_values().to_frame('missing_value').reset_index(),
         big_df.dtypes.to_frame('feature_type').reset_index(),
         on = 'index',
         how = 'inner')

In [12]:
df_dtypes.sort_values(['missing_value', 'feature_type'])

,index,missing_value,feature_type
0,Airline,0,object
1,Date_of_Journey,0,object
2,Source,0,object
3,Destination,0,object
4,Dep_Time,0,object
5,Arrival_Time,0,object
6,Duration,0,object
7,Additional_Info,0,object
8,Route,1,object
9,Total_Stops,1,object


# Remove Constant Feature

In [13]:
def find_constant_features(dataFrame):
    const_features = []
    for column in list(dataFrame.columns):
        if dataFrame[column].unique().size < 2:
            const_features.append(column)
    return const_features

In [14]:
const_features = find_constant_features(big_df)

In [15]:
const_features

[]

# Drop Duplicate Rows

In [16]:
big_df.drop_duplicates(inplace= True)

# Remove Duplicate Columns

In [17]:
def duplicate_columns(frame):
    groups = frame.columns.to_series().groupby(frame.dtypes).groups
    dups = []

    for t, v in groups.items():

        cs = frame[v].columns
        vs = frame[v]
        lcs = len(cs)

        for i in range(lcs):
            ia = vs.iloc[:,i].values
            for j in range(i+1, lcs):
                ja = vs.iloc[:,j].values
                if np.array_equal(ia, ja):
                    dups.append(cs[i])
                    break
    return dups

In [18]:
duplicate_cols = duplicate_columns(big_df)

In [19]:
duplicate_cols

[]

# Feature Engineering

In [20]:
big_df['Date']=big_df['Date_of_Journey'].str.split('/').str[0]
big_df['Month']=big_df['Date_of_Journey'].str.split('/').str[1]
big_df['Year']=big_df['Date_of_Journey'].str.split('/').str[2]

In [21]:
big_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,24,03,2019
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,1,05,2019
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0,9,06,2019
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,12,05,2019
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,01,03,2019


In [22]:
big_df['Date']=big_df['Date'].astype(int)
big_df['Month']=big_df['Month'].astype(int)
big_df['Year']=big_df['Year'].astype(int)

In [23]:
big_df=big_df.drop(['Date_of_Journey'],axis=1)

In [24]:
big_df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,24,3,2019
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,1,5,2019
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0,9,6,2019
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,12,5,2019
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,1,3,2019


In [25]:
big_df['Arrival_Time']=big_df['Arrival_Time'].str.split(' ').str[0]

In [26]:
big_df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10,2h 50m,non-stop,No info,3897.0,24,3,2019
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,1,5,2019
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25,19h,2 stops,No info,13882.0,9,6,2019
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,12,5,2019
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,1,3,2019


In [27]:
big_df['Arrival_Hour'] = big_df['Arrival_Time'] .str.split(':').str[0]
big_df['Arrival_Minute'] = big_df['Arrival_Time'] .str.split(':').str[1]

In [28]:
big_df.head(2)

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10,2h 50m,non-stop,No info,3897.0,24,3,2019,01,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15


In [29]:
big_df=big_df.drop(['Arrival_Time'],axis=1)

In [30]:
big_df.head(3)

,Airline,Source,Destination,Route,Dep_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,2h 50m,non-stop,No info,3897.0,24,3,2019,01,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,19h,2 stops,No info,13882.0,9,6,2019,04,25


In [31]:
big_df['Dep_Hour'] = big_df['Dep_Time'] .str.split(':').str[0]
big_df['Dep_Minute'] = big_df['Dep_Time'] .str.split(':').str[1]

In [32]:
big_df.head(2)

,Airline,Source,Destination,Route,Dep_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,2h 50m,non-stop,No info,3897.0,24,3,2019,01,10,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15,05,50


In [33]:
big_df=big_df.drop(['Dep_Time'],axis=1)

In [34]:
big_df.head(2)

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897.0,24,3,2019,01,10,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15,05,50


In [35]:
big_df[big_df['Total_Stops'].isnull()]

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute
9039,Air India,Delhi,Cochin,NaN,23h 40m,NaN,No info,7480.0,6,5,2019,09,25,09,45


In [36]:
big_df['Total_Stops']=big_df['Total_Stops'].fillna('1 stop')

In [37]:
big_df['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', '4 stops'],
      dtype=object)

In [38]:
big_df['Total_Stops']=big_df['Total_Stops'].replace('non-stop','0 stop')

In [39]:
big_df['Total_Stops'].unique()

array(['0 stop', '2 stops', '1 stop', '3 stops', '4 stops'], dtype=object)

In [40]:
big_df['Stop'] = big_df['Total_Stops'].str.split(' ').str[0]

In [41]:
big_df.head(2)

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute,Stop
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,0 stop,No info,3897.0,24,3,2019,01,10,22,20,0
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15,05,50,2


In [42]:
big_df=big_df.drop(['Total_Stops'],axis=1)

In [43]:
big_df=big_df.drop(['Additional_Info'],axis=1)

In [44]:
big_df.head(3)

,Airline,Source,Destination,Route,Duration,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute,Stop
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,3897.0,24,3,2019,01,10,22,20,0
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,7662.0,1,5,2019,13,15,05,50,2
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,13882.0,9,6,2019,04,25,09,25,2


In [45]:
big_df['Stop']=big_df['Stop'].astype(int)

In [46]:
big_df.dtypes

Airline            object
Source             object
Destination        object
Route              object
Duration           object
Price             float64
Date                int32
Month               int32
Year                int32
Arrival_Hour       object
Arrival_Minute     object
Dep_Hour           object
Dep_Minute         object
Stop                int32
dtype: object

In [47]:
big_df['Arrival_Hour']=big_df['Arrival_Hour'].astype(int)
big_df['Arrival_Minute']=big_df['Arrival_Minute'].astype(int)
big_df['Dep_Hour']=big_df['Dep_Hour'].astype(int)
big_df['Dep_Minute']=big_df['Dep_Minute'].astype(int)

In [48]:
big_df.dtypes

Airline            object
Source             object
Destination        object
Route              object
Duration           object
Price             float64
Date                int32
Month               int32
Year                int32
Arrival_Hour        int32
Arrival_Minute      int32
Dep_Hour            int32
Dep_Minute          int32
Stop                int32
dtype: object

In [49]:
big_df['Route_1'] = big_df['Route'] .str.split('→ ').str[0]
big_df['Route_2'] = big_df['Route'] .str.split('→ ').str[1]
big_df['Route_3'] = big_df['Route'] .str.split('→ ').str[2]
big_df['Route_4'] = big_df['Route'] .str.split('→ ').str[3]
big_df['Route_5'] = big_df['Route'] .str.split('→ ').str[4]

In [50]:
big_df['Route_1'].fillna("None",inplace = True)
big_df['Route_2'].fillna("None",inplace = True)
big_df['Route_3'].fillna("None",inplace = True)
big_df['Route_4'].fillna("None",inplace = True)
big_df['Route_5'].fillna("None",inplace = True)

In [51]:
big_df=big_df.drop(['Route'], axis=1)
big_df=big_df.drop(['Duration'], axis=1)

In [52]:
big_df.head()

,Airline,Source,Destination,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute,Stop,Route_1,Route_2,Route_3,Route_4,Route_5
0,IndiGo,Banglore,New Delhi,3897.0,24,3,2019,1,10,22,20,0,BLR,DEL,None,None,None
1,Air India,Kolkata,Banglore,7662.0,1,5,2019,13,15,5,50,2,CCU,IXR,BBI,BLR,None
2,Jet Airways,Delhi,Cochin,13882.0,9,6,2019,4,25,9,25,2,DEL,LKO,BOM,COK,None
3,IndiGo,Kolkata,Banglore,6218.0,12,5,2019,23,30,18,5,1,CCU,NAG,BLR,None,None
4,IndiGo,Banglore,New Delhi,13302.0,1,3,2019,21,35,16,50,1,BLR,NAG,DEL,None,None


In [53]:
big_df['Airline'].unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [54]:
big_df['Price'].fillna((big_df['Price'].mean()), inplace=True)

In [55]:
big_df.head()

,Airline,Source,Destination,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute,Stop,Route_1,Route_2,Route_3,Route_4,Route_5
0,IndiGo,Banglore,New Delhi,3897.0,24,3,2019,1,10,22,20,0,BLR,DEL,None,None,None
1,Air India,Kolkata,Banglore,7662.0,1,5,2019,13,15,5,50,2,CCU,IXR,BBI,BLR,None
2,Jet Airways,Delhi,Cochin,13882.0,9,6,2019,4,25,9,25,2,DEL,LKO,BOM,COK,None
3,IndiGo,Kolkata,Banglore,6218.0,12,5,2019,23,30,18,5,1,CCU,NAG,BLR,None,None
4,IndiGo,Banglore,New Delhi,13302.0,1,3,2019,21,35,16,50,1,BLR,NAG,DEL,None,None


In [56]:
big_df.tail()

,Airline,Source,Destination,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute,Stop,Route_1,Route_2,Route_3,Route_4,Route_5
2666,Air India,Kolkata,Banglore,9026.642454,6,6,2019,20,25,20,30,1,CCU,DEL,BLR,None,None
2667,IndiGo,Kolkata,Banglore,9026.642454,27,3,2019,16,55,14,20,0,CCU,BLR,None,None,None
2668,Jet Airways,Delhi,Cochin,9026.642454,6,3,2019,4,25,21,50,1,DEL,BOM,COK,None,None
2669,Air India,Delhi,Cochin,9026.642454,6,3,2019,19,15,4,0,1,DEL,BOM,COK,None,None
2670,Multiple carriers,Delhi,Cochin,9026.642454,15,6,2019,19,15,4,55,1,DEL,BOM,COK,None,None


# Feature Encoding

In [57]:
#Converting the Categorical into integer variable

In [58]:
from sklearn.preprocessing import LabelEncoder

In [59]:
lb_encode = LabelEncoder()
big_df["Airline"] = lb_encode.fit_transform(big_df["Airline"])
big_df["Destination"] = lb_encode.fit_transform(big_df["Destination"])
big_df["Source"] = lb_encode.fit_transform(big_df["Source"])
big_df['Route_1']= lb_encode.fit_transform(big_df["Route_1"])
big_df['Route_2']= lb_encode.fit_transform(big_df["Route_2"])
big_df['Route_3']= lb_encode.fit_transform(big_df["Route_3"])
big_df['Route_4']= lb_encode.fit_transform(big_df["Route_4"])
big_df['Route_5']= lb_encode.fit_transform(big_df["Route_5"])

In [60]:
big_df.head()

,Airline,Source,Destination,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute,Stop,Route_1,Route_2,Route_3,Route_4,Route_5
0,3,0,5,3897.0,24,3,2019,1,10,22,20,0,0,13,24,12,4
1,1,3,0,7662.0,1,5,2019,13,15,5,50,2,2,25,1,3,4
2,4,2,1,13882.0,9,6,2019,4,25,9,25,2,3,32,4,5,4
3,3,3,0,6218.0,12,5,2019,23,30,18,5,1,2,34,3,12,4
4,3,0,5,13302.0,1,3,2019,21,35,16,50,1,0,34,8,12,4


In [61]:
df_dtypes = pd.merge(big_df.isnull().sum(axis = 0).sort_values().to_frame('missing_value').reset_index(),
         big_df.dtypes.to_frame('feature_type').reset_index(),
         on = 'index',
         how = 'inner')

In [62]:
df_dtypes.sort_values(['missing_value', 'feature_type'])

,index,missing_value,feature_type
0,Airline,0,int32
1,Route_3,0,int32
2,Route_2,0,int32
3,Route_1,0,int32
4,Stop,0,int32
5,Dep_Minute,0,int32
6,Dep_Hour,0,int32
7,Route_4,0,int32
8,Arrival_Minute,0,int32
9,Year,0,int32


# Divide the data set into test and train

In [63]:
df_train = big_df[0:10683]
df_test = big_df[10683:]
df_test = df_test.drop(['Price'], axis =1)

In [64]:
df_train = big_df[0:10683]
df_test = big_df[10683:]
df_test = df_test.drop(['Price'], axis =1)



In [65]:
X = big_df.drop(['Price'], axis=1)
y = big_df.Price

In [66]:
X.head()

,Airline,Source,Destination,Date,Month,Year,Arrival_Hour,Arrival_Minute,Dep_Hour,Dep_Minute,Stop,Route_1,Route_2,Route_3,Route_4,Route_5
0,3,0,5,24,3,2019,1,10,22,20,0,0,13,24,12,4
1,1,3,0,1,5,2019,13,15,5,50,2,2,25,1,3,4
2,4,2,1,9,6,2019,4,25,9,25,2,3,32,4,5,4
3,3,3,0,12,5,2019,23,30,18,5,1,2,34,3,12,4
4,3,0,5,1,3,2019,21,35,16,50,1,0,34,8,12,4


In [67]:
y.shape

(13108,)

In [68]:
y.head()

0     3897.0
1     7662.0
2    13882.0
3     6218.0
4    13302.0
Name: Price, dtype: float64

In [69]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=None)

# Build our model method

In [70]:
ytrain.head()

2503    13377.000000
230     14641.000000
5338    10577.000000
1785     9026.642454
2554     7480.000000
Name: Price, dtype: float64

In [71]:
from scipy import stats
from scipy.stats import norm, skew
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn import svm
from lightgbm import LGBMRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split,GridSearchCV, RandomizedSearchCV
#from sklearn.metrics import accuracy_score,cohen_kappa_score,confusion_matrix,mean_absolute_error,mean_squared_error
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.svm import SVR

In [72]:
transformer = RobustScaler().fit(xtrain)
scale_xtrain=transformer.transform(xtrain)

In [73]:
trans = RobustScaler().fit(xtest)
scale_xtest=trans.transform(xtest)

In [79]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators =[100, 200, 300, 400,600,800,1000]
# Number of features to consider at every split
max_features = [2,3,4,6,8,10,12,14,18]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [100, 200, 300, 400, 600, 800, 1000], 'max_features': [2, 3, 4, 6, 8, 10, 12, 14, 18], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6], 'bootstrap': [True, False]}


In [80]:
r_f = RandomForestRegressor()

In [81]:
rf_random = RandomizedSearchCV(estimator = r_f, param_distributions = random_grid, n_iter = 4,
                               cv =5, verbose=2, random_state=42, n_jobs = -1)

In [82]:
rf_random.fit(scale_xtrain,ytrain)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.0min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...


In [83]:
y_preds = rf_random.predict(scale_xtest)
from sklearn.metrics import explained_variance_score
print(explained_variance_score(ytest, y_preds))

0.42169146167918536


In [127]:
random_grid_svr = {'kernel':['poly'],
               'C':[10, 20, 100, 200,400,600,800,1000],
               'gamma':['auto'],                                     
              }
svm=SVR()

In [128]:
rf_random_svr = RandomizedSearchCV(estimator =svm, param_distributions = random_grid_svr, n_iter = 5,
                               cv =5, verbose=3, random_state=42, n_jobs = -1)


In [129]:
rf_random_svr.fit(scale_xtrain,ytrain)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   33.1s finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                                 epsilon=0.1, gamma='auto_deprecated',
                                 kernel='rbf', max_iter=-1, shrinking=True,
                                 tol=0.001, verbose=False),
                   iid='warn', n_iter=5, n_jobs=-1,
                   param_distributions={'C': [10, 20, 100, 200, 400, 600, 800,
                                              1000],
                                        'gamma': ['auto'], 'kernel': ['poly']},
                   pre_dispatch='2*n_jobs', random_state=42, refit=True,
                   return_train_score=False, scoring=None, verbose=3)

In [130]:
y_preds_svr = rf_random_svr.predict(scale_xtest)

In [131]:
print(explained_variance_score(ytest, y_preds_svr))

0.3593841719998828


In [141]:
from sklearn import metrics
def print_accuracy_report(y_test, y_pred,X_test, model):
 print('R Squared(Accuracy)', metrics.r2_score(y_test, y_pred))   
 print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
 print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
 print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
 print('Root Mean Squared Log Error',np.sqrt(mean_squared_log_error( y_test, y_pred )))

In [142]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import accuracy_score
from xgboost import XGBRegressor
params = {
  'reg_alpha':[1e-5]
}
def XGBoostRegressorModel(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42, test_size=0.3)
    rf=XGBRegressor(learning_rate =0.1, n_estimators=240,
     subsample=0.8, colsample_bytree=0.8,
     scale_pos_weight=1, seed=42, max_depth= 10, min_child_weight=5,gamma=0.0, reg_alpha=1e-5)
    rf_random = GridSearchCV(estimator = rf, param_grid = params, cv = 3, verbose=2, n_jobs = 8)
    rf_random.fit(X_train, y_train)
    print(rf_random.best_params_)
    y_pred = rf_random.best_estimator_.predict(X_test)
    print(print_accuracy_report(y_test, y_pred, X_test, rf_random.best_estimator_))
    return rf_random.best_estimator_
xgBoostModel = XGBoostRegressorModel(X,y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   3 | elapsed:    7.4s finished


[16:11:23] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
{'reg_alpha': 1e-05}
R Squared(Accuracy) 0.5855916263901877
Mean Absolute Error: 1853.2822684489047
Mean Squared Error: 7178903.3757308
Root Mean Squared Error: 2679.34756530966
Root Mean Squared Log Error 0.29903660046847436
None


# LGBMRegressor Hyperparamiter Tuning